In [2]:
import pandas as pd

In [3]:
df_train = pd.read_csv("data/hackathon_loreal_train_set.csv")
df_train

,Unnamed: 0,text,skincare,hair,make-up,other,index
0,0,Skeeterbytes wrote: Bassam Guy wrote: I don't ...,0.0,0.0,0.0,1.0,0
1,1,Refueling before the long week 🍵 - You’re mean...,0.0,0.0,0.0,1.0,1
2,2,Just in case no one has told you today YOU ARE...,0.0,0.0,1.0,0.0,2
3,3,Do not forget to get some sunshine safely if y...,0.0,0.0,0.0,1.0,3
4,4,Abstract Background Humans have dramatically c...,0.0,0.0,0.0,1.0,4
...,...,...,...,...,...,...,...
10567,13209,☑ Day: 273 Outifit: Thor (Gold Foil) Back Blin...,0.0,0.0,0.0,1.0,13209
10568,13210,In order to study the various trends and patte...,0.0,0.0,0.0,1.0,13210
10569,13212,This brand has not yet registered with Influen...,0.0,0.0,0.0,1.0,13212
10570,13213,here is my regime get rid of any acne scars an...,1.0,0.0,0.0,0.0,13213


# watch data

In [4]:
df_train.columns

Index(['Unnamed: 0', 'text', 'skincare', 'hair', 'make-up', 'other', 'index'], dtype='object')

In [5]:
df = df_train[df_train['Unnamed: 0'] != df_train['index']]
df

,Unnamed: 0,text,skincare,hair,make-up,other,index


# delete one columns

In [6]:
df_train = df_train.drop(['Unnamed: 0'], axis = 1)
df_train

,text,skincare,hair,make-up,other,index
0,Skeeterbytes wrote: Bassam Guy wrote: I don't ...,0.0,0.0,0.0,1.0,0
1,Refueling before the long week 🍵 - You’re mean...,0.0,0.0,0.0,1.0,1
2,Just in case no one has told you today YOU ARE...,0.0,0.0,1.0,0.0,2
3,Do not forget to get some sunshine safely if y...,0.0,0.0,0.0,1.0,3
4,Abstract Background Humans have dramatically c...,0.0,0.0,0.0,1.0,4
...,...,...,...,...,...,...
10567,☑ Day: 273 Outifit: Thor (Gold Foil) Back Blin...,0.0,0.0,0.0,1.0,13209
10568,In order to study the various trends and patte...,0.0,0.0,0.0,1.0,13210
10569,This brand has not yet registered with Influen...,0.0,0.0,0.0,1.0,13212
10570,here is my regime get rid of any acne scars an...,1.0,0.0,0.0,0.0,13213


# preprocessing one text

In [7]:
import pandas as pd

import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.metrics import ConfusionMatrix
from nltk.stem.snowball import SnowballStemmer

from nltk import word_tokenize, WordNetLemmatizer, PorterStemmer
from nltk import pos_tag
from nltk import ngrams
from nltk import sent_tokenize

from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Thomas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
Onetext = df_train['text'].iloc[1]
Onetext

'Refueling before the long week 🍵 - You’re meant to Drink it, but I prefer it in my 🍵 From to , matcha is a natural powder of freshly ground leaves! ✅thanks to its numerous antioxidant benefits! ✅Including it in your beauty routine requires very little time and effort to see benefits ✅ The best part of it is that it’s already in your cupboard and doubles as a delicious drink? It really doesn’t get better than this! 🛒 Matcha tea purchased from my favorite online store faithfultonature'

In [9]:
OnetextLower = Onetext.lower()
OnetextLower

'refueling before the long week 🍵 - you’re meant to drink it, but i prefer it in my 🍵 from to , matcha is a natural powder of freshly ground leaves! ✅thanks to its numerous antioxidant benefits! ✅including it in your beauty routine requires very little time and effort to see benefits ✅ the best part of it is that it’s already in your cupboard and doubles as a delicious drink? it really doesn’t get better than this! 🛒 matcha tea purchased from my favorite online store faithfultonature'

In [10]:
tokenizeSingletext = word_tokenize(OnetextLower)
tokenizeSingletext

['refueling',
 'before',
 'the',
 'long',
 'week',
 '🍵',
 '-',
 'you',
 '’',
 're',
 'meant',
 'to',
 'drink',
 'it',
 ',',
 'but',
 'i',
 'prefer',
 'it',
 'in',
 'my',
 '🍵',
 'from',
 'to',
 ',',
 'matcha',
 'is',
 'a',
 'natural',
 'powder',
 'of',
 'freshly',
 'ground',
 'leaves',
 '!',
 '✅thanks',
 'to',
 'its',
 'numerous',
 'antioxidant',
 'benefits',
 '!',
 '✅including',
 'it',
 'in',
 'your',
 'beauty',
 'routine',
 'requires',
 'very',
 'little',
 'time',
 'and',
 'effort',
 'to',
 'see',
 'benefits',
 '✅',
 'the',
 'best',
 'part',
 'of',
 'it',
 'is',
 'that',
 'it',
 '’',
 's',
 'already',
 'in',
 'your',
 'cupboard',
 'and',
 'doubles',
 'as',
 'a',
 'delicious',
 'drink',
 '?',
 'it',
 'really',
 'doesn',
 '’',
 't',
 'get',
 'better',
 'than',
 'this',
 '!',
 '🛒',
 'matcha',
 'tea',
 'purchased',
 'from',
 'my',
 'favorite',
 'online',
 'store',
 'faithfultonature']

In [11]:
lemmatizer = WordNetLemmatizer()
lemmatizeSingleText = list(map(lemmatizer.lemmatize, tokenizeSingletext))
lemmatizeSingleText

['refueling',
 'before',
 'the',
 'long',
 'week',
 '🍵',
 '-',
 'you',
 '’',
 're',
 'meant',
 'to',
 'drink',
 'it',
 ',',
 'but',
 'i',
 'prefer',
 'it',
 'in',
 'my',
 '🍵',
 'from',
 'to',
 ',',
 'matcha',
 'is',
 'a',
 'natural',
 'powder',
 'of',
 'freshly',
 'ground',
 'leaf',
 '!',
 '✅thanks',
 'to',
 'it',
 'numerous',
 'antioxidant',
 'benefit',
 '!',
 '✅including',
 'it',
 'in',
 'your',
 'beauty',
 'routine',
 'requires',
 'very',
 'little',
 'time',
 'and',
 'effort',
 'to',
 'see',
 'benefit',
 '✅',
 'the',
 'best',
 'part',
 'of',
 'it',
 'is',
 'that',
 'it',
 '’',
 's',
 'already',
 'in',
 'your',
 'cupboard',
 'and',
 'double',
 'a',
 'a',
 'delicious',
 'drink',
 '?',
 'it',
 'really',
 'doesn',
 '’',
 't',
 'get',
 'better',
 'than',
 'this',
 '!',
 '🛒',
 'matcha',
 'tea',
 'purchased',
 'from',
 'my',
 'favorite',
 'online',
 'store',
 'faithfultonature']

In [12]:
stop_punctuation = [':', '(', ')', '/', '|', ',', ']', ';',
                    '.', '*', '#', '"', '&', '~', '``',
                    '-', '_', '\\', '@','?','!','\'', '[', '🍵']
TextCleanPonctuation = [word for word in lemmatizeSingleText if word not in stop_punctuation]
TextCleanPonctuation

['refueling',
 'before',
 'the',
 'long',
 'week',
 'you',
 '’',
 're',
 'meant',
 'to',
 'drink',
 'it',
 'but',
 'i',
 'prefer',
 'it',
 'in',
 'my',
 'from',
 'to',
 'matcha',
 'is',
 'a',
 'natural',
 'powder',
 'of',
 'freshly',
 'ground',
 'leaf',
 '✅thanks',
 'to',
 'it',
 'numerous',
 'antioxidant',
 'benefit',
 '✅including',
 'it',
 'in',
 'your',
 'beauty',
 'routine',
 'requires',
 'very',
 'little',
 'time',
 'and',
 'effort',
 'to',
 'see',
 'benefit',
 '✅',
 'the',
 'best',
 'part',
 'of',
 'it',
 'is',
 'that',
 'it',
 '’',
 's',
 'already',
 'in',
 'your',
 'cupboard',
 'and',
 'double',
 'a',
 'a',
 'delicious',
 'drink',
 'it',
 'really',
 'doesn',
 '’',
 't',
 'get',
 'better',
 'than',
 'this',
 '🛒',
 'matcha',
 'tea',
 'purchased',
 'from',
 'my',
 'favorite',
 'online',
 'store',
 'faithfultonature']

In [13]:
stoplist = stopwords.words('english')
TextCleanPonctuationWord = [word for word in TextCleanPonctuation if word not in stoplist]
TextCleanPonctuationWord

['refueling',
 'long',
 'week',
 '’',
 'meant',
 'drink',
 'prefer',
 'matcha',
 'natural',
 'powder',
 'freshly',
 'ground',
 'leaf',
 '✅thanks',
 'numerous',
 'antioxidant',
 'benefit',
 '✅including',
 'beauty',
 'routine',
 'requires',
 'little',
 'time',
 'effort',
 'see',
 'benefit',
 '✅',
 'best',
 'part',
 '’',
 'already',
 'cupboard',
 'double',
 'delicious',
 'drink',
 'really',
 '’',
 'get',
 'better',
 '🛒',
 'matcha',
 'tea',
 'purchased',
 'favorite',
 'online',
 'store',
 'faithfultonature']

# functions

In [14]:
def preprocessing(singleText):
    lemmatizer = WordNetLemmatizer()
    stop_punctuation = [':', '(', ')', '/', '|', ',', ']', ';',
                    '.', '*', '#', '"', '&', '~', '``',
                    '-', '_', '\\', '@','?','!','\'', '[']
    stoplist = stopwords.words('english')

    LowerSingleText = singleText.lower()
    tokenizeSingletext = word_tokenize(LowerSingleText)
    lemmatizeSingleText = list(map(lemmatizer.lemmatize, tokenizeSingletext))
    
    TextCleanPonctuation = [word for word in lemmatizeSingleText if word not in stop_punctuation and word not in stoplist]

    return TextCleanPonctuation

def clean_text(text):
    text_preprocessed = preprocessing(text)
    
    clean_mail = [word for word in text_preprocessed if word not in stop_punctuation and not word in stoplist]
    return ' '.join(clean_mail)

In [15]:
result = clean_text(df_train["text"].iloc[0])
result

"skeeterbytes wrote bassam guy wrote n't anything longer 60mm except $ 99 plastic fantastic '' 40-150. 've never really tried avian photography would like get decent shot unlike opportunity arises used c-af+tr retrospect since turkey lawyer refused fly despite plea would s-af done better mf peaking magnify exposed lawyer dressed dark suit bright cloudy sky meteorologist say smoke remnant ca northern va wa +2 +2.5 ev much encouraged chroma 'm great judging distance would say wa 35+ meter away pretty intense purple fringing lightroom ha effective one-click remover 'm sure software ha hard control know panasonic 7-14 4 never use c-af+tr preferring standard c-af s-af instance s-af fine attorney n't going anywhere pay bill get s-af+magnify allow tweak focus plus want stop lens bit sharpen except 7-14 nothing else f2.8 5.6 seems stopped 'll watch tried magnify wa shaky long fl 'll try lower scale factor also raise shutter speed freeze flapping 're iso200 's good lens within realm 40-150 pro 

# On applique cela pour toute la base de donnée

In [16]:
df_train["clean_content"] = df_train.text.apply(clean_text)
df_train["clean_content"]

0        skeeterbytes wrote bassam guy wrote n't anythi...
1        refueling long week ’ meant drink prefer match...
2        case one ha told today strong beautiful love 🌹...
3        forget get sunshine safely ☀️ head link bio se...
4        abstract background human dramatically changed...
                               ...                        
10567    ☑ day 273 outifit thor gold foil back bling et...
10568    order study various trend pattern prevailing c...
10569    brand ha yet registered influenster work brand...
10570    regime get rid acne scar get way smoother heal...
10571    's true used work caddie trump national nj don...
Name: clean_content, Length: 10572, dtype: object

# vectorization

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

## separate

In [18]:
X = df_train['clean_content']
Y = df_train.drop(['clean_content', 'index'], axis = 1)
X

0        skeeterbytes wrote bassam guy wrote n't anythi...
1        refueling long week ’ meant drink prefer match...
2        case one ha told today strong beautiful love 🌹...
3        forget get sunshine safely ☀️ head link bio se...
4        abstract background human dramatically changed...
                               ...                        
10567    ☑ day 273 outifit thor gold foil back bling et...
10568    order study various trend pattern prevailing c...
10569    brand ha yet registered influenster work brand...
10570    regime get rid acne scar get way smoother heal...
10571    's true used work caddie trump national nj don...
Name: clean_content, Length: 10572, dtype: object

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=4)

In [20]:
pipe = make_pipeline(CountVectorizer(), TfidfTransformer())
pipe.fit(x_train)
feat_train = pipe.transform(x_train)
feat_train

<8457x71330 sparse matrix of type '<class 'numpy.float64'>'
	with 704785 stored elements in Compressed Sparse Row format>

In [21]:
feat_train

<8457x71330 sparse matrix of type '<class 'numpy.float64'>'
	with 704785 stored elements in Compressed Sparse Row format>

# Model

In [22]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=50)
clf.fit(feat_train, y_train)

TypeError: '<' not supported between instances of 'float' and 'str'

In [ ]:
feat_test = pipe.transform(x_test)
clf.score(feat_test, y_test) 

# visualisation result

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
score = clf.predict_proba(feat_test)
score
#fpr, tpr, th = roc_curve(y_list_test, score[:, 1])

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=(4,4))
ax.plot([0, 1], [0, 1], 'k--')
aucf = auc(fpr, tpr)
ax.plot(fpr, tpr, label='auc=%1.5f' % aucf)
ax.set_title('Courbe ROC - classifieur de sentiments')
ax.legend();

In [23]:
import re
def emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [24]:
df_train.text[1]

'Refueling before the long week 🍵 - You’re meant to Drink it, but I prefer it in my 🍵 From to , matcha is a natural powder of freshly ground leaves! ✅thanks to its numerous antioxidant benefits! ✅Including it in your beauty routine requires very little time and effort to see benefits ✅ The best part of it is that it’s already in your cupboard and doubles as a delicious drink? It really doesn’t get better than this! 🛒 Matcha tea purchased from my favorite online store faithfultonature'

In [25]:
emoji(df_train.text[1])

'Refueling before the long week  - You’re meant to Drink it, but I prefer it in my  From to , matcha is a natural powder of freshly ground leaves! thanks to its numerous antioxidant benefits! Including it in your beauty routine requires very little time and effort to see benefits  The best part of it is that it’s already in your cupboard and doubles as a delicious drink? It really doesn’t get better than this!  Matcha tea purchased from my favorite online store faithfultonature'

In [26]:
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
# Function for removing emoticons
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

In [27]:
remove_emoticons(df_train.text[1])

'Refueling before the long week 🍵 - You’re meant to Drink it, but I prefer it in my 🍵 From to , matcha is a natural powder of freshly ground leaves! ✅thanks to its numerous antioxidant benefits! ✅Including it in your beauty routine requires very little time and effort to see benefits ✅ The best part of it is that it’s already in your cupboard and doubles as a delicious drink? It really doesn’t get better than this! 🛒 Matcha tea purchased from my favorite online store faithfultonature'

In [28]:
import re
try:
    import cPickle as pickle
except ImportError:
    import pickle  

with open('Emoticon_Dict.p', 'rb') as fp:
    Emoticon_Dict = pickle.load(fp)

def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in Emoticon_Dict) + u')')
    return emoticon_pattern.sub(r'', text)

In [29]:
with open('Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(text):
    for emot in Emoji_Dict:
        text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
    return text

In [30]:
convert_emojis_to_word(df_train.text[1])

'Refueling before the long week teacup_without_handle - You’re meant to Drink it, but I prefer it in my teacup_without_handle From to , matcha is a natural powder of freshly ground leaves! white_heavy_check_markthanks to its numerous antioxidant benefits! white_heavy_check_markIncluding it in your beauty routine requires very little time and effort to see benefits white_heavy_check_mark The best part of it is that it’s already in your cupboard and doubles as a delicious drink? It really doesn’t get better than this! shopping_cart Matcha tea purchased from my favorite online store faithfultonature'

In [ ]:
lipstick
man_getting_haircut
man_getting_haircut_dark_skin_tone
man_getting_haircut_light_skin_tone
man_getting_haircut_medium-dark_skin_tone
man_getting_haircut_medium-light_skin_tone
man_getting_haircut_medium_skin_tone
man_getting_massage
man_getting_massage_dark_skin_tone
man_getting_massage_light_skin_tone
man_getting_massage_medium-dark_skin_tone
man_getting_massage_medium-light_skin_tone
man_getting_massage_medium_skin_tone
nail_polish
nail_polish_dark_skin_tone
nail_polish_light_skin_tone
nail_polish_medium-dark_skin_tone
nail_polish_medium-light_skin_tone
nail_polish_medium_skin_tone
woman_getting_haircut
woman_getting_haircut_dark_skin_tone
woman_getting_haircut_light_skin_tone
woman_getting_haircut_medium-dark_skin_tone
woman_getting_haircut_medium-light_skin_tone
woman_getting_haircut_medium_skin_tone
woman_getting_massage
woman_getting_massage_dark_skin_tone
woman_getting_massage_light_skin_tone
woman_getting_massage_medium-dark_skin_tone
woman_getting_massage_medium-light_skin_tone
woman_getting_massage_medium_skin_tone

In [31]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [32]:
def test_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    if re.search(emoji_pattern, string):
        return 1
    else: return 0
    
compteur = 0
for i in df_train.index:
    compteur += test_emoji(df_train.text.iloc[i])

print("Nombre de posts avec des emojis :", compteur, "|| Pourcentage (%) :", (compteur/df_train.shape[0])*100)

Nombre de posts avec des emojis : 3282 || Pourcentage (%) : 31.04426787741203


In [33]:
list_categories = ["skincare", "hair", "make-up", "other"]

for category in list_categories:
    print("Nombre de posts de catégorie", category, ":", len(df_train[df_train[category] == 1]),
          "|| Pourcentage (%) :", (len(df_train[df_train[category] == 1]) / df_train.shape[0]) * 100 )

Nombre de posts de catégorie skincare : 1303 || Pourcentage (%) : 12.325009458948164
Nombre de posts de catégorie hair : 1272 || Pourcentage (%) : 12.031782065834278
Nombre de posts de catégorie make-up : 1907 || Pourcentage (%) : 18.038214150586455
Nombre de posts de catégorie other : 6667 || Pourcentage (%) : 63.06280741581536


In [ ]:
df_train.text.apply(convert_emojis_to_word)

In [35]:
def test_url(string):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    if re.search(url_pattern, string):
        return 1
    else: return 0
    
compteur_2 = 0
for i in df_train.index:
    compteur_2 += test_url(df_train.text.iloc[i])

print("Nombre de posts avec des urls :", compteur_2, "|| Pourcentage (%) :", (compteur_2/df_train.shape[0])*100)

Nombre de posts avec des urls : 21 || Pourcentage (%) : 0.19863791146424517


In [38]:
def test_s(string):
    url_pattern = re.compile(r"--")
    if re.search(url_pattern, string):
        return 1
    else: return 0
    
compteur_3 = 0
for i in df_train.index:
    compteur_3 += test_s(df_train.text.iloc[i])

print("Nombre de posts avec des 's :", compteur_3, "|| Pourcentage (%) :", (compteur_3/df_train.shape[0])*100)

Nombre de posts avec des 's : 420 || Pourcentage (%) : 3.9727582292849033


In [41]:
df_train.text[0]

'Skeeterbytes wrote: Bassam Guy wrote: I don\'t have anything longer than 60mm except my $99 "plastic [not so] fantastic" 40-150. I\'ve never really tried avian photography before but would like to get some decent shots (unlike these) when the opportunity arises. I used C-AF+TR. In retrospect, since the Turkey Lawyer refused to fly despite my pleas. Would S-AF have done better? MF, Peaking or Magnify or both? I exposed for the lawyer, dressed in dark suits against a bright cloudy sky (some meteorologists say it is smoke remnants from CA in Northern VA). Was +2 and +2.5 EV too much and encouraged the chroma? I\'m not great at judging distance but I would say I was about 35+ meters away. Pretty intense purple fringing. Lightroom has an effective one-click remover and I\'m sure other software has, too. That should not be hard to control. I know how to do that. I own the Panasonic 7-14 4 Never use C-AF+tr myself, preferring standard C-AF or S-AF. In this instance, S-AF is fine because your